In [56]:
import games
import time
import random
from collections import namedtuple, Counter, defaultdict
from notebook import psource, pseudocode
import functools 
cache = functools.lru_cache(10**6)
from utils import argmax

* `actions(self, state)`: Given a game state, this method generates all the legal actions possible from this state, as a list or a generator. Returning a generator rather than a list has the advantage that it saves space and you can still operate on it as a list.


* `result(self, state, move)`: Given a game state and a move, this method returns the game state that you get by making that move on this game state.


* `utility(self, state, player)`: Given a terminal game state and a player, this method returns the utility for that player in the given terminal game state. While implementing this method assume that the game state is a terminal game state. The logic in this module is such that this method will be called only on terminal game states.


* `terminal_test(self, state)`: Given a game state, this method should return `True` if this game state is a terminal state, and `False` otherwise.


* `to_move(self, state)`: Given a game state, this method returns the player who is to play next. This information is typically stored in the game state, so all this method does is extract this information and return it.


* `display(self, state)`: This method prints/displays the current state of the game.

In [57]:
class Mancala(games.Game):
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def __init__(self, board = [4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0], player_turn=True):
        #4 marbels per cup, on each player's side
        self.board = board
        #True for player 1 and False for player 2
        self.player_turn = player_turn
        self.initial = games.GameState(to_move=player_turn, utility=0, board=board, moves=self.get_moves(player_turn,self.board))
        
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
#         print("To move:",state.to_move)
        return self.get_moves(state.to_move, state.board)

    def get_moves(self, player_turn, cups):
        moves = []
#         print("To move:",bool(xplayer_turn))
        if bool(player_turn):
            board_range = range(0,6)
        else:
            board_range = range(7,13)
        for m in board_range:
            if cups[m] != 0 and m%7!=6:
                moves += [m]
#         print("Moves:",moves)
        return moves

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        #getting the marbles in the cup
        board = state.board[:]
        marblesInCup = board[move]
        currentPlayer = state.to_move
        
        if currentPlayer:
            scoreSpot = 6
        else:
            scoreSpot = 13

        board[move]=0
        initialPlace = move
        for i in range(0,marblesInCup):
            boardIndex = move%14
            #Makes sure that the players don't score on the opponant score board
            if currentPlayer==True and boardIndex==12:
                move+=1
            elif currentPlayer==False and boardIndex==5:
                move+=1
            #initial cup will always be empty & skipped over
            elif initialPlace == boardIndex+1:
                move+=1
            move+=1
            board[move%14]+=1

        lastSpot = move%14
        #If the last piece you drop is in an empty hole on your side,
        #you capture that piece and any pieces in the hole directly opposite.
#         print(lastSpot)
#         print("Initial board:",board)
        
        if board[lastSpot] == 1:
            if (not currentPlayer and lastSpot>6 and lastSpot<13):
                
                board[0:6] = reversed(board[0:6])
                board[scoreSpot] += board[(lastSpot+7)%14]
                board[(lastSpot+7)%14]=0
                board[0:6] = reversed(board[0:6])
                board[scoreSpot]+=board[lastSpot]
                board[lastSpot]=0
            elif (currentPlayer and lastSpot < 6):
                board[7:13] = reversed(board[7:13])
                board[scoreSpot] += board[(lastSpot+7)%14]
                board[(lastSpot+7)%14]=0
                board[7:13] = reversed(board[7:13])
                board[scoreSpot]+=board[lastSpot]

        #the player changes only when the current player's marble don't land on their score cup
        if (currentPlayer and lastSpot!=6) or (not currentPlayer and lastSpot!=13):
            currentPlayer = not currentPlayer
#         print("Next player:",bool(currentPlayer))
        return games.GameState(to_move=currentPlayer,
                         utility=self.utility(state,currentPlayer),
                         board=board, moves=self.get_moves(currentPlayer,state.board))
#         return state

    def utility(self, state, player):
        """Return the value of this final state to player."""
        sum = 0
        if (state.board[0] == 0 and state.board[1] == 0 and state.board[2] == 0 and state.board[3] == 0 and state.board[4] == 0 and state.board[5] == 0) :
            for i in range(7,13) :
                sum += state.board[i]
                state.board[i]=0
            state.board[13] += sum
#             self.display(state)

        elif (state.board[7] == 0 and state.board[8] == 0 and state.board[9] == 0 and state.board[10] == 0 and state.board[11] == 0 and state.board[12] == 0) :
            for i in range(6) :
                sum += state.board[i]
                state.board[i]=0
            state.board[6] += sum
#             self.display(state)
        
        
        return self.check_winner(state.board)
        
    def check_winner(self, board) :
        """Checks which player won. Player 1 wins if it returns 1, Player 2 wins if it returns 2."""
        if board[6] > board[13] :
            return 1
        elif board[6] < board[13] :
            return -1
        elif board[6] == board[13] :
            return 0

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        
        if (state.board[0] == 0 and state.board[1] == 0 and state.board[2] == 0 and state.board[3] == 0 and state.board[4] == 0 and state.board[5] == 0) or (state.board[7] == 0 and state.board[8] == 0 and state.board[9] == 0 and state.board[10] == 0 and state.board[11] == 0 and state.board[12] == 0) :
            return True
        else :
            return False

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
#         print(state)
        print("In display:",state.board)
#         self.__repr__(state)
        board = "Player 2"
        board += "\n\t6\t5\t4\t3\t2\t1"
        board += "\n------------------------------------------------------------\n"
        board += str(state.board[13]) + "\t"
        for elem in range(12,6,-1):
            board += str(state.board[elem]) + "\t"
        board += "\n\t"
        for elem in range(0,6):
            board += str(state.board[elem]) + "\t"
        board += str(state.board[6])
        board += "\n------------------------------------------------------------"
        board += "\n\t1\t2\t3\t4\t5\t6\n"
        board += "Player 1\n"    
        print(board)
#     def __repr__(self):
#         return self.board

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        curr_players = players
        while True:
            for player in curr_players:
                curr_move = state.to_move
                move = player(self, state)
                
                state = self.result(state, move)
#                 print("Player in play_games:",curr_move)
#                 print("Moves in play_game:",state.moves)
                if self.terminal_test(state):
                    return self.utility(state, self.to_move(self.initial))
                #making sure the same player can go again if they have another chance to
                if state.to_move==curr_move:
                    if state.to_move == False:
                        curr_players = players[::-1] 
                    else:
                        curr_players = players
                    break
        
    
    def play_game_2(game, strategies: dict, verbose=False):
    
        state = game.initial
        while not game.is_terminal(state):
            player = state.to_move
            move = strategies[player](game, state)
            state = game.result(state, move)
            if verbose: 
                print('Player', player, 'move:', move)
                print(state)
        return state

In [59]:
def h1(state): 
    if state.to_move==True:
        return state.board[6]
    else:
        return state.board[13]
    
def h2(state):
    sum =0
    if state.to_move==True:
         for i in range(6) :
                sum += state.board[i]
    else:
        for i in range(7,13) :
                sum += state.board[i]
    return sum
    
def alphabeta_player_h1(game, state):
    return games.alphabeta_cutoff_search(state, game,8,None,h1)

def alphabeta_player_h2(game, state):
    return games.alphabeta_cutoff_search(state, game,8,None,h2)

def monte_carlo(game,state):
    best_action = None
    moves = state.moves
    current_player = state.to_move
    board = state.board
    if len(moves)==1:
        best_action = moves[0]
    elif len(moves)>1:
        num_wins=[0]*len(moves)
        for i in moves:
            cur_moves = moves[:].remove(i)
            for _ in range(int(len(moves)/100)):
                random_move = random.choice(cur_moves)
                newBoard = game.result(state, random_move)
                while not game.terminal_test(newBoard):
                    cur_moves = cur_moves.remove(random_move)
                    random_move = random.choice(cur_moves)
                    newBoard = game.result(state, random_move)
                if (game.utility(newBoard)==1 and current_player) or (game.utility(newBoard)==-1 and not current_player):
                    num_wins[i]+=1
        best_action = moves[num_wins.index(max(num_wins))]   
    return best_action

In [61]:
mancalaPlay_1 = Mancala()
wins, loss, tie, avr = 0,0,0,0.0
searches = tuple((games.random_player,alphabeta_player_h1, alphabeta_player_h2, monte_carlo))
for player_1 in searches:
    for player_2 in searches:
        wins, loss, tie, avr = 0,0,0,0.0
        for i in range(50):
            start_time = time.time()
            value = mancalaPlay_1.play_game(player_1, player_2)
            avr+= time.time()-start_time
            if value==1:
                wins+=1
            if value==0:
                tie+=1
            if value==-1:
                loss+=1
        print("{} vs {}".format(str(player_1.__name__),str(player_2.__name__)))
        print('Wins: {}; Loss: {}; Ties:  {}; Time: {}'.format(wins/50,tie/50,loss/50,(avr/50)))

random_player vs random_player
Wins: 0.5; Loss: 0.02; Ties:  0.48; Time: 0.0004916000366210938
random_player vs alphabeta_player_h1
Wins: 0.4; Loss: 0.04; Ties:  0.56; Time: 1.269310369491577
random_player vs alphabeta_player_h2
Wins: 0.86; Loss: 0.04; Ties:  0.1; Time: 4.672917971611023
random_player vs monte_carlo
Wins: 0.66; Loss: 0.02; Ties:  0.32; Time: 0.00039159297943115235
alphabeta_player_h1 vs random_player
Wins: 0.84; Loss: 0.0; Ties:  0.16; Time: 1.6260629177093506
alphabeta_player_h1 vs alphabeta_player_h1
Wins: 0.0; Loss: 0.0; Ties:  1.0; Time: 2.7637949562072754
alphabeta_player_h1 vs alphabeta_player_h2
Wins: 1.0; Loss: 0.0; Ties:  0.0; Time: 5.535833134651184
alphabeta_player_h1 vs monte_carlo
Wins: 1.0; Loss: 0.0; Ties:  0.0; Time: 1.8351396322250366
alphabeta_player_h2 vs random_player
Wins: 0.74; Loss: 0.02; Ties:  0.24; Time: 5.667612829208374
alphabeta_player_h2 vs alphabeta_player_h1
Wins: 0.0; Loss: 0.0; Ties:  1.0; Time: 9.156083445549012
alphabeta_player_h2 vs

In [62]:
random_player_time = (0.0004916000366210938+1.269310369491577+4.672917971611023+0.00039159297943115235)/4.0
alphabeta_player_h1_time = (1.6260629177093506+2.7637949562072754+5.535833134651184+1.8351396322250366)/4.0
alphabeta_player_h2_time= (5.667612829208374+9.156083445549012+9.88056279182434+3.5662796306610107)/4.0
monte_carlo_time = (0.00029148578643798826+1.1509992361068726+4.667851114273072+0.00018285274505615236)/4.0

print("Random Player time:",random_player_time)
print("H1 Player time:",alphabeta_player_h1_time)
print("H2 Player time:",alphabeta_player_h2_time)
print("Monte Carlo time:",monte_carlo_time)

Random Player time: 1.4857778835296631
H1 Player time: 2.9402076601982117
H2 Player time: 7.067634674310685
Monte Carlo time: 1.4548311722278597


In [63]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)
    
def report(game, searchers):
    for searcher in searchers:
        game = CountCalls(game)
        start_time = time.time()
        searcher(game, game.initial)
        diff = time.time()-start_time
        print('Result states: {:7,d}; Terminal tests: {:7,d}; time taken:  {} ,for {}'.format(
            game._counts['result'], game._counts['terminal_test'],diff, searcher.__name__))
        
report(Mancala(), (games.random_player,alphabeta_player_h1, alphabeta_player_h2, monte_carlo))

Result states:       0; Terminal tests:       0; time taken:  0.00022077560424804688 ,for random_player
Result states:  49,353; Terminal tests:  16,877; time taken:  0.4712550640106201 ,for alphabeta_player_h1
Result states: 160,213; Terminal tests:  43,805; time taken:  1.670335054397583 ,for alphabeta_player_h2
Result states:       0; Terminal tests:       0; time taken:  3.2901763916015625e-05 ,for monte_carlo
